**Rationale** Hypothesis testing is foundational to the entire field of statistics. Without the Central Limit Theorem and hypothesis testing, we would not have modern day (social) science. In marketing, this is generally most important when doing A/B testing of ads. In this assignment, you will practice computing basic summary statistics and conducting hypothesis tests. 

[Datasets](https://drive.google.com/drive/folders/1D6D-zRU3oiP12c0dFMQjs66JDhBZBtax?usp=sharing) required:
1. [FB ad campaign data](https://drive.google.com/file/d/1JfwvumxS2oys8ZcQtsb8GGmeRfnkzFdy/view?usp=sharing).
1. [Starbucks Promos](https://drive.google.com/file/d/1W7x5_PU5KszT8mqXqqEmRfXuR8CRr1DX/view?usp=sharing)

# 1. (2 points) FB Ad campaigns

1. Use a groupby operation create a dataframe called `sumstats` consisting of the total impressions and clicks for each `xyz_campaign_id`.
1. Using `sumstats`, create a new column called `ctr` that represents the [click through rate](https://support.google.com/google-ads/answer/2615875?hl=en) for each campaign. Which campaign had the highest CTR?
1. Create a column for `sd` for standard deviation of the click through rate. 

Hint: Recall from the notes that the standard deviation of a binomial distribution with $Pr(Success) = p$ is $SD(p) = \sqrt{p(1-p)}$. So if the probability of a binary outcome (such as clicking on an ad) is observed to be .3, the standard deviation is $\sqrt{.3(.7)}=\sqrt{.21}$.

In [ ]:
# imports and mount google drive
import os, pandas as pd, numpy as np
from scipy import stats

from google.colab import drive
drive.mount('drive')

Mounted at drive


In [ ]:
# set the path to the datasets for A5
fpath = '/content/drive/MyDrive/DataSetsForA5'
os.listdir(fpath)

['facebook_ads.csv', 'starbucks_promos.csv']

In [ ]:
# read in the facebook_ads.csv file as the dataframe variable named fb
fb = pd.read_csv('/content/drive/MyDrive/DataSetsForA5/facebook_ads.csv')


In [ ]:
# Take a look at the first 5 rows
fb.head(5)
# Observe that each row represents 1 ad from 1 campaign and a target demographic group (age, gender, interest)


,ad_id,xyz_campaign_id,fb_campaign_id,age,gender,interest,Impressions,Clicks,Spent,Total_Conversion,Approved_Conversion
0,708746,916,103916,30-34,M,15,7350,1,1.43,2,1
1,708749,916,103917,30-34,M,16,17861,2,1.82,2,0
2,708771,916,103920,30-34,M,20,693,0,0.00,1,0
3,708815,916,103928,30-34,M,28,4259,1,1.25,1,0
4,708818,916,103928,30-34,M,28,4133,1,1.29,1,1


In [ ]:
# use a groupby to create new dataframe, sumstats that tabulates the total clicks and impressions for each campaign.
# Generic groupby syntax: sumstats = df.groupby('groupbyvariable')[['summary_variable1', 'summary_variable2']].summaryfunction().reset_index()
# use the correct dataframe name, variables (i.e. column names) and summaryfunction
#sumstats = fb.groupby(['Clicks', 'Impressions']).sum().reset_index()
sumstats = fb.groupby('xyz_campaign_id')[['Clicks', 'Impressions']].sum().reset_index()

sumstats

,xyz_campaign_id,Clicks,Impressions
0,916,113,482925
1,936,1984,8128187
2,1178,36068,204823716


In [ ]:
# create a new column in sumstats called 'ctr' (short for click through rate)
# recall that ctr = clicks / impressions
sumstats['ctr'] = sumstats.Clicks / sumstats.Impressions


In [ ]:
# take a look at the whole sumstats dataframe
sumstats


,xyz_campaign_id,Clicks,Impressions,ctr
0,916,113,482925,0.000234
1,936,1984,8128187,0.000244
2,1178,36068,204823716,0.000176


In [ ]:
# create a new column ctr_sd for the std. dev of the ctr (See hint above)
# pay attention to PEMDAS, esp. that you want to take the sqrt after computing p*(1-p)
# np.sqrt(x) returns the sqrt of some number x
# alternatively, x**.5 also returns the sqrt of x
sumstats['ctr_sd'] = abs(sumstats.ctr - sumstats.ctr.mean())**2
sumstats['ctr_sd'] = np.sqrt((sumstats.ctr_sd))

sumstats


,xyz_campaign_id,Clicks,Impressions,ctr,ctr_sd
0,916,113,482925,0.000234,0.000016
1,936,1984,8128187,0.000244,0.000026
2,1178,36068,204823716,0.000176,0.000042


In [ ]:
# take another look at the whole sumstats dataframe
# compare the ctr_std to calculating it by hand, make sure you did this correctly.



## Answer (edit this cell)
The campaign with the highest CTR is: **where xyz_Compaign_id = 916**


# 2. (4 points) Hypothesis testing and confidence intervals

1. Compute the 95% confidence interval for the CTR for each campaign. Compare the confidence intervals of campaign 916 and 936. What can you conclude about the relative perfrmance of the 2 ads in the population (e.g. are they very different, similar, etc.)? How about 916 vs. 1178?
1. Was campaign 936 statistically different compared to campaign 916? How about 936 vs. 1178? Use the `ttest_2sample` function from the notes. Remember, you must define it in your Colab session in order to use it (execute the cell w/ the function).

Given these statistical tests what would you recommend in terms of allocation of the ad budget? 

In [ ]:
# use stats.norm.ppf([.025, .975], mean, std error)
# where mean is sample mean and std error is std dev / sqrt(obs) (from Central Limit Theorem)

# confidence interval for ad 916

stats.norm.ppf([.025, .975], sumstats.ctr.mean(), sumstats.ctr_sd[0])



array([0.00018683, 0.00024929])

In [ ]:
# confidence interval for ad 936

stats.norm.ppf([.025, .975], sumstats.ctr.mean(), sumstats.ctr_sd[1])

array([0.00016704, 0.00026908])

In [ ]:
# confidence interval for ad 1178

stats.norm.ppf([.025, .975], sumstats.ctr.mean(), sumstats.ctr_sd[2])

array([0.00013581, 0.00030031])

**Edit this cell**

1. (write 1-2 sentences to compare the confidence interval of 916 vs 936's CTR)

- Confidence interval of 916 vs 936's CTR are respectively [0.00018683, 0.00024929], [0.00016704, 0.00026908] both have very small difference. 



1. (write 1-2 sentences to compare the confidence interval of 916 vs 1178's CTR)

- Confidence interval of 916 vs 1178's CTR are respectively [0.00018683, 0.00024929], [0.00013581, 0.00030031]. It shows a big diffrence than above comparison. 

In [ ]:
# Use this custom funciton

def ttest_2sample(m1,sd1,N1,m2,sd2,N2, twotail = True, equalvar = False):
    """
    This function requires you tu supply:
    m1: mean of sample 1
    sd1: std. dev of sample 1
    N1: number of obs of sample 1
    m2: mean of sample 2
    sd2: std dev of sample 2
    N2: number of obs of sample 2

    Optional inputs:
    twotail = True (default) / False. If False, then 1 tail
    equalvar = True / False (default). If True, assumes equal population variance.
    """

    # The difference between equal and unequal variance is only in how to compute
    # the test statistic and degree of freedom.
    if equalvar:
        spsquare = ((N1-1)*sd1**2+(N2-1)*sd2**2)/(N1+N2-2) 
        T = (m1-m2)/np.sqrt(spsquare*(1/N1+1/N2))
        nu = N1+N2-2
    else:
        nu = (sd1**2/N1+sd2**2/N2)**2/((sd1**2/N1)**2/(N1-1)+(sd2**2/N2)**2/(N2-1)) # new degree of freedom
        T = (m1-m2)/(np.sqrt(sd1**2/N1+sd2**2/N2))
    
    # If the first mean is bigger, we need to do 1- cdf
    # Otherwise just compute cdf
    if m1>m2:
        pval = 1-stats.t.cdf(T, df = nu)
    else:
        pval = stats.t.cdf(T, df = nu)

    # return p values
    # If 2 tail, we must multiply by 2
    # otherwise we just return the computed pval
    if twotail == True:
        return pval*2
    else:
        return pval

In [ ]:
# run a 2 tail 2 sample t-test to compare the population CTRs for 936 and 916
ttest_2sample(sumstats.ctr.mean(),sumstats.ctr_sd[1],2,sumstats.ctr.mean(),sumstats.ctr_sd[0],2, twotail = True, equalvar = False)

1.0

In [ ]:
# run a 2 tail 2 sample t-test to compare the population CTRs for 936 and 1178
ttest_2sample(sumstats.ctr.mean(),sumstats.ctr_sd[1],2,sumstats.ctr.mean(),sumstats.ctr_sd[2],2, twotail = True, equalvar = False)

1.0

**Edit this cell to answer**

(Write a few sentences to describe how to proceed with the allocation of the ad budget going forward. Be sure to use the 2 tail t-tests above to support your strategy)

- As above valuation shows both the adds comsumed same amount of budget. Result of 2 tail t-test for both comparison resulted the same value. 

# 3. (4 points)

Using the Starbucks dataset, conduct an appropriate statistical test to determine:
1. Is there any statistically appreciable difference in the redemption rates of BOGO vs. Discount promotions?
1. Among those who redeemed an offer and reported their income, is there any statistical difference in the average income associated with BOGO vs. discount redemptions? Note that not every observation reports income (select only the observations that do).

In [ ]:
# read in the starbucks_promos.csv file as the dataframe variable named sb, use index_col = 0 option

sb = pd.read_csv('/content/drive/MyDrive/DataSetsForA5/starbucks_promos.csv', index_col = 0)


In [ ]:
# preview first 5 rows to get a sense of the contents of the dataframe
# note that each observation represents an offer received by a customer
sb.head(5)


,uid,event,time,gender,age,register_date,income,offer_id,offer_reward,channels,difficulty,duration,offer_type,offer_time,transaction_amount,redeem_time,redeemed
1,0020c2b971eb4e9188eac86d93036a77,offer received,0,F,59,20160304,90000.0,fafdcd668e3743c1bb461111dcafc2a4,2.0,"['web', 'email', 'mobile', 'social']",10.0,240.0,discount,0.0,17.63,54.0,1
4,005500a7188546ff8a767329a2f7c76a,offer received,0,M,56,20171209,47000.0,ae264e3637204a6fb9bb56bc8210ddfd,10.0,"['email', 'mobile', 'social']",10.0,168.0,bogo,0.0,NaN,NaN,0
5,0056df74b63b4298809f0b375a304cf4,offer received,0,M,54,20160821,91000.0,9b98b8c7a33c4b65b9aebfe6a799e6d9,5.0,"['web', 'email', 'mobile']",5.0,168.0,bogo,0.0,27.86,132.0,1
6,00715b6e55c3431cb56ff7307eb19675,offer received,0,F,58,20171207,119000.0,ae264e3637204a6fb9bb56bc8210ddfd,10.0,"['email', 'mobile', 'social']",10.0,168.0,bogo,0.0,27.26,12.0,1
8,00840a2ca5d2408e982d56544dc14ffd,offer received,0,M,26,20141221,61000.0,2906b810c7d4411798c6938adc9daaa5,2.0,"['web', 'email', 'mobile']",10.0,168.0,discount,0.0,6.05,540.0,1


In [ ]:
# select the reemed column for discount offers, name this data the variable disc

disc = sb.redeemed
disc



1        1
4        0
5        1
6        1
8        1
        ..
76272    0
76273    1
76274    0
76275    1
76276    1
Name: redeemed, Length: 61042, dtype: int64

In [ ]:
# select the reemed column for buy one get one (bogo) offers, name this data the variable bogo
bogo = (sb[sb['offer_type'] == "bogo"]).redeemed
#bogo = sb[offer_type == 'bogo'].redeemed
bogo

4        0
5        1
6        1
9        0
10       0
        ..
76269    0
76271    1
76273    1
76274    0
76275    1
Name: redeemed, Length: 30499, dtype: int64

In [ ]:
# use stats.ttest_ind to test whether the 2 types of offers yield different redemption rates
stats.ttest_ind(disc, bogo)


Ttest_indResult(statistic=9.943880291425248, pvalue=2.7557298896958967e-23)

In [ ]:
stats.ttest_ind(bogo, disc, equal_var=False)

Ttest_indResult(statistic=-9.918368694263158, pvalue=3.6076261223079263e-23)

In [ ]:
# Select the income column for those who redeemed the BOGO offer, call this the variable bogo_income
# note you need to use 3 condtitions for the row selection: one for redeemed, one for bogo offer, and one for income is not missing
# FYI: dataframe.somecolumn.notnull() is the Boolean evaluation for whether the somecolumn is not missing data

Condition1 = sb[sb['offer_type'] == "bogo"]
Condition2 = (Condition1[Condition1['redeemed'] == 1]).income
bogo_income = Condition2.notnull()
bogo_income


5        True
6        True
11       True
23       True
24       True
         ... 
76263    True
76268    True
76271    True
76273    True
76275    True
Name: income, Length: 16395, dtype: bool

In [ ]:
# Select the income column for those who redeemed the discount offer, call this the variable disc_income
# note you need to use 3 condtitions for the row selection: one for redeemed, one for discount offer,  and one for income is not missing
# FYI: dataframe.somecolumn.notnull() is the Boolean evaluation for whether the somecolumn is not missing data

Condition1 = sb[sb['offer_type'] == "discount"]
Condition2 = (Condition1[Condition1['redeemed'] == 1]).income
dics_income = Condition2.notnull()
dics_income


1        True
8        True
13       True
17       True
21       True
         ... 
76256    True
76259    True
76264    True
76266    True
76276    True
Name: income, Length: 18530, dtype: bool

In [ ]:
# What is the average income for bogo redeemers (bogo_income.mean())?

bogo_income.mean()

0.9718206770356816

In [ ]:
# What is the average income for discount redeemers?

dics_income.mean()

0.9557474365893146

In [ ]:
# use stats.ttest_ind to test whether the average income of the redeemers of the 2 offer types are different

stats.ttest_ind(bogo_income, dics_income)

Ttest_indResult(statistic=7.979152697919397, pvalue=1.5181115688426378e-15)

In [ ]:
stats.ttest_ind(bogo_income, dics_income, equal_var=False)


Ttest_indResult(statistic=8.084212390978582, pvalue=6.458516385412022e-16)

**Edit this cell**
1. (Write a sentence to describe the relative redemption rates of the 2 offer types based on the result of the t-test).
- stats.ttest_ind(disc, bogo) gives the result Ttest_indResult(statistic=9.943880291425248, pvalue=2.7557298896958967e-23)

1. (Write a sentence to describe the relative incomes of the redeemers of the 2 offer types based on the result of the t-test).
- stats.ttest_ind(bogo_income, dics_income)  gives the result Ttest_indResult(statistic=7.979152697919397, pvalue=1.5181115688426378e-15)